In [1]:
from kevin_utils2 import *
from keras.layers.merge import dot, add, concatenate
from __future__ import division, print_function
import gzip, os

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield eval(l)

In [3]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Beauty.json.gz')

In [4]:
df=df.dropna(axis=1,how='any')

In [5]:
df.head()

,reviewerID,asin,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A39HTATAQ9V7YF,0205616461,"[0, 0]",i do love this moisturizer and would recommend...,5.0,bio-active anti-aging serum,1369699200,"05 28, 2013"
1,A3JM6GV9MNOF9X,0558925278,"[0, 1]",I received this product before the deadline.I ...,3.0,"This product is ok, I'm use Baby Kabuki in moment",1355443200,"12 14, 2012"
2,A1Z513UWSAAO0F,0558925278,"[0, 0]",I love this set. Great buy for the price. I do...,5.0,I love this set,1404691200,"07 7, 2014"
3,A1WMRR494NWEWV,0733001998,"[0, 0]","A nice moisturizer, all natural ingredients an...",4.0,Nice Moisturizer,1382572800,"10 24, 2013"
4,A3IAAVS479H7M7,0737104473,"[2, 2]",Please research the MAC Hello Kitty collection...,1.0,Fake MAC,1274227200,"05 19, 2010"


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2023070 entries, 0 to 2023069
Data columns (total 8 columns):
reviewerID        object
asin              object
helpful           object
reviewText        object
overall           float64
summary           object
unixReviewTime    int64
reviewTime        object
dtypes: float64(1), int64(1), object(6)
memory usage: 138.9+ MB


#create embeddings

In [7]:
users = df.reviewerID.unique()
prodID = df.asin.unique()

In [8]:
userid2idx = {o:i for i,o in enumerate(users)}
prodid2idx = {o:i for i,o in enumerate(prodID)}

In [9]:
df.asin = df.asin.apply(lambda x: prodid2idx[x])
df.reviewerID = df.reviewerID.apply(lambda x: userid2idx[x])

In [10]:
n_users = df.reviewerID.nunique()
n_asin = df.asin.nunique()
n_users, n_asin

(1210271, 249274)

##taking a sample

In [11]:
n_factors = 50
np.random.seed = 42

In [12]:
n = len(df)
n

2023070

In [13]:
samp_size = 200000

In [14]:
idxs = sorted(np.random.choice(n, samp_size, replace=False))

In [15]:
sample_df = df.iloc[idxs]

In [16]:
train_ratio = 0.8
train_size = int(samp_size * train_ratio)

In [17]:
train_size

160000

In [18]:
val = sample_df[train_size:]
trn = sample_df[:train_size]
len(val), len(trn)

(40000, 160000)

In [19]:
#neural network

In [20]:
from keras.regularizers import l2

In [21]:
def embedding_input(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg))(inp)

In [22]:
user_in, u = embedding_input('reviewerID', n_users, n_factors, 1e-4)
n_asin, m = embedding_input('asin', n_asin, n_factors, 1e-4)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(1210271, 50, input_length=1, embeddings_regularizer=<keras.reg...)`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(249274, 50, input_length=1, embeddings_regularizer=<keras.reg...)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [23]:

#x=merge([u,m], mode='concat')
x = dot([u, m], axes=(2,2))
x = Flatten()(x)
x = Dropout(0.3)(x)
#x = Dropout(0.02)(x)
x = Dense(1000, activation='relu')(x)
x = Dense(500, activation='relu')(x)
x = Dropout(0.75)(x)
x = Dense(1)(x)
nn = Model([user_in, n_asin], x)
nn.compile(Adam(0.001), loss='mse')

In [24]:
nn.fit([trn.reviewerID, trn.asin], trn.overall, batch_size=64, epochs=8, 
          validation_data=([val.reviewerID, val.asin], val.overall))

Train on 160000 samples, validate on 40000 samples
Epoch 1/8
160000/160000 [==============================] - 321s 2ms/step - loss: 2.1180 - val_loss: 1.7100
Epoch 2/8
160000/160000 [==============================] - 316s 2ms/step - loss: 1.9293 - val_loss: 1.7103
Epoch 3/8
160000/160000 [==============================] - 316s 2ms/step - loss: 1.8687 - val_loss: 1.7409
Epoch 4/8
160000/160000 [==============================] - 316s 2ms/step - loss: 1.8203 - val_loss: 1.7099
Epoch 5/8
160000/160000 [==============================] - 316s 2ms/step - loss: 1.7873 - val_loss: 1.7105
Epoch 6/8
160000/160000 [==============================] - 316s 2ms/step - loss: 1.7697 - val_loss: 1.7099
Epoch 7/8
160000/160000 [==============================] - 316s 2ms/step - loss: 1.7583 - val_loss: 1.7130
Epoch 8/8
160000/160000 [==============================] - 316s 2ms/step - loss: 1.7491 - val_loss: 1.7143


In [30]:
nn.save_weights('bias.1.7')

In [31]:
nn.load_weights('bias.1.7')

In [32]:
nn.predict([np.array([4]), np.array([6])])

array([[ 4.0534]], dtype=float32)